In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [7]:
df = pd.read_csv('~/Documents/GitHub/2022-Pergi-Kuliner-Restaurant-Data/pergikuliner_restaurants_dataset.csv')
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,Momo Paradise,Tipe KulinerJepang,"\n Mal Kelapa Gading 5, Lanta...",https://pergikuliner.com/restaurants/jakarta/m...,Cabang: ya,"""Tunai,Visa,Master,Debet""",<bound method PageElement.get_text of <div cla...,\n Di atas Rp. 200.000 /orang...,<bound method PageElement.get_text of <time it...,"100% Halal,Ruang VIP,Bisa reservasi,Area parki...","Wifi,Pesan antar,Area merokok,Area outdoor,24 ...",021 45853709,4.71,"4.9,4.5,4.9,4.7,4.6"
1,Caspar,Tipe KulinerSpanyol,\n Sudirman Suites Apartment ...,https://pergikuliner.com/restaurants/jakarta/c...,"PembayaranTunai, Debet","""Tunai,Debet""",NaN,\n Di atas Rp. 200.000 /orang...,<bound method PageElement.get_text of <time it...,"Wifi,Area merokok,Area outdoor,100% Halal,Bisa...","Pesan antar,24 Jam,Ruang VIP,Vegetarian",082217817880,4.58,"4.7,4.8,4.8,4.7"
2,Furusato Izakaya,Tipe KulinerJepang,\n Sudirman Suites Apartment ...,https://pergikuliner.com/restaurants/jakarta/f...,"PembayaranTunai, Visa, Master, Debet","""Tunai,Visa,Master,Debet""",NaN,\n Di atas Rp. 200.000 /orang...,<bound method PageElement.get_text of <time it...,"Wifi,Area merokok,Area outdoor,Ruang VIP,Bisa ...","Pesan antar,24 Jam,100% Halal,Vegetarian",08119250020,4.47,"4.5,4.8,4.0,4.6,4.5"
3,Osteria Gia,Tipe KulinerItalia,"\n Pacific Place Mall, Lantai...",https://pergikuliner.com/restaurants/jakarta/o...,Cabang: ya,"""Tunai,Visa,Master,Debet""",NaN,\n Di atas Rp. 200.000 /orang...,<bound method PageElement.get_text of <time it...,"Wifi,Area merokok,Bisa reservasi,Area parkir,A...","Pesan antar,Area outdoor,24 Jam,100% Halal,Rua...",021 5153300,4.45,"4.6,4.5,4.1,4.5,4.5"
4,Isshin,Tipe KulinerJepang,"\n Central Park, Lantai Lower...",https://pergikuliner.com/restaurants/jakarta/i...,Cabang: ya,"""Tunai,Visa,Master,Debet,Go-pay""",NaN,\n Rp. 100.000 - Rp. 200.000 ...,<bound method PageElement.get_text of <time it...,"Wifi,100% Halal,Bisa reservasi,Area parkir","Pesan antar,Area merokok,Area outdoor,24 Jam,R...",021 56985157,4.46,"4.7,4.4,4.1,4.7,4.5"


In [ ]:
#cleared
#col 1, 2, 3, 9, 10 and 12 do not have nans.
#col 5: cleared. look precise enough, removed the quotation marks.

#needing rechecks
#col 11: 57 phone number nans. check if they really don't have phone numbers.

#awaiting script rerun
#col 6: promotional texts does not get scraped correctly. added str() to get_text() method but the script needs to be rerun.
#col 8: schedule does not get scraped correctly. added str() to get_text() method but the script needs to be rerun.

#reengineer
#col 4: sometimes branch data does not exist and the columns got filled with other information. put 'kapasitas kursi' in the right cols.
#col 13: bot does not scrape rating when it goes below 4.

In [36]:
#checks
df['5'].str.replace('"', '').str.split(',', expand=True).stack().value_counts()

Tunai     1480
Debet     1370
Visa      1225
Master    1225
Ovo        395
Go-pay     376
Dana       221
dtype: int64

In [38]:
df['4'].value_counts()

Cabang: ya                                                 966
PembayaranTunai, Visa, Master, Debet                       255
PembayaranTunai                                             39
PembayaranTunai, Visa, Master, Debet, Go-pay, Ovo, Dana     36
PembayaranTunai, Debet                                      32
PembayaranTunai, Visa, Master, Debet, Go-pay, Ovo           32
PembayaranTunai, Debet, Go-pay, Ovo, Dana                   13
Kapasitas:  50 kursi                                        12
PembayaranTunai, Visa, Master, Debet, Ovo                   11
Kapasitas:  30 kursi                                         7
PembayaranTunai, Visa, Master, Debet, Go-pay                 6
PembayaranTunai, Debet, Go-pay, Ovo                          6
Kapasitas:  100 kursi                                        5
PembayaranTunai, Go-pay, Ovo, Dana                           5
Kapasitas:  70 kursi                                         5
Kapasitas:  20 kursi                                   

In [37]:
df['5'] = df['5'].str.replace('"', '')

In [23]:
['Name', 'culinary_type', 'address', 'url', '']

['Name', 'culinary_type', 'address', 'url', '']